In [1]:
# conclusion
# did we answer the problem statement/purpose
# what can be improved in this process
# what are the flaws of my model

# Production Model and Conclusion

---

- bar chart of scores of different models
- select last non ridge/lasso model
- explain why this model over others
- higher r squared score by a bit
- maintains interpretability of coeficients
    - interpret coef